## Is this just magic?  What is Numba doing to make code run quickly?

Let's define a trivial example function.

In [ ]:
from numba import jit

In [ ]:
@jit
def add(a, b):
    return a + b

In [ ]:
add(1, 1)

Numba examines Python bytecode and then translates this into an 'intermediate representation'.  To view this IR, run (compile) `add` and you can access the `inspect_types` method.

In [ ]:
add.inspect_types()

Ok.  Numba is has correctly inferred the type of the arguments, defining things as `int64` and running smoothly.  

(What happens if you do `add(1., 1.)` and then `inspect_types`?)

In [ ]:
add(1., 1.)

In [ ]:
add.inspect_types()

### What about the actual LLVM code?

You can see the actual LLVM code generated by Numba using the `inspect_llvm()` method.  Since it's a `dict`, doing the following will be slightly more visually friendly.

In [ ]:
for k, v in add.inspect_llvm().items():
    print(k, v)

## But there's a caveat

Now, watch what happens when we try to do something that is natural in Python, but not particularly mathematically sound:

In [ ]:
def add_strings(a, b):
    return a + b

In [ ]:
add_strings_jit = jit()(add_strings)

In [ ]:
add_strings_jit('a', 'b')

It worked, but what does `inspect_types` tell us?

In [ ]:
add_strings_jit.inspect_types()

## What's all this pyobject business?  

This means it has been compiled in `object` mode.  This can be a faster than regular python if it can do loop lifting, but not that fast.  
We want those `pyobjects` to be `int64` or another type that can be inferred by Numba. Your best bet is forcing `nopython` mode: this will throw an error if Numba finds itself in object mode, so that you _know_ that it can't give you speed.

For the full list of supported Python and NumPy features in `nopython` mode, see the Numba documentation here: http://numba.pydata.org/numba-doc/latest/reference/pysupported.html

## Figuring out what isn't working

In [ ]:
%%file nopython_failure.py
from numba import jit

@jit
def add(a, b):
    for i in range(100):
        c = i
        f = i + 7
        l = c + f
        
    return a + b

add('a', 'b')

In [ ]:
!numba --annotate-html fail.html nopython_failure.py

[fail.html](fail.html)

## Forcing `nopython` mode

In [ ]:
add_strings_jit = jit(nopython=True)(add_strings)

In [ ]:
add_strings_jit('a', 'b')

In [ ]:
from numba import njit

In [ ]:
add_strings_jit = njit(add_strings)

In [ ]:
add_strings_jit('a', 'b')

## Other compilation flags

There are two other main compilation flags for `@jit`

```python
cache=True
```

if you don't want to always want to get dinged by the compilation time for every run. This will actually save the compiled function into something like a `pyc` file in your `__pycache__` directory, so even between sessions you should have nice fast performance.

```python
nogil=True
```

This releases the GIL.  Note, however, that it doesn't do anything else, like make your program threadsafe.  You have to manage all of those things on your own (use `concurrent.futures`).